In [357]:
from lxml import etree as et
from sklearn.metrics import classification_report
import numpy
import os
import pandas as pd
import json
import time

from utils import *

DEBUG = False

In [358]:
def get_pmid(docs):
    documents = [
        document.split("/")[-1] for document in docs
    ]
    return documents

In [359]:
def get_answers(answers_files):
    all_answers = dict()
    for a in answers_files:
        with open(a, "r") as f:
            d = json.loads(f.read())
            if DEBUG:
                print(f"{len(d)} answers found in {a}")
            for key, value in d.items():
                if key in all_answers.keys():
                    print(f"MULTIPLE ANSWERS FOR {key}")
                if isinstance(value,list):
                    all_answers[key] = value[0] # get the first value which is answer not prediction for the yes/no
                else:
                    all_answers[key] = value
    return all_answers

def get_three_files(a_dir):
    return [
        a_dir + "/factoid/predictions.json",
        a_dir + "/list/predictions.json",
        a_dir + "/yesno/predictions.json",
    ]

def get_col_list(gold_df,gen_df,col):
    gold_col = gold_df.loc[:,['id',col]].copy()
    gen_col = gen_df.loc[:,['id',col]].copy() 
    
    gold = gold_col.to_dict(orient='list')
    gen = gen_col.to_dict(orient='list')
    gen_ids = gen['id']
    gen_vals = gen[col]
    gold_ids = gold['id']
    gold_vals = gold[col]
    return gold_ids,gold_vals,gen_ids,gen_vals

In [360]:

def parse_xml(xml_file, dir_for_qa): 
    no_answers = 0
    # get answers
    qa_answers = get_answers(get_three_files(dir_for_qa))
    # get ir and qu
    df_cols = ['id','human_concepts','documents','full_abstracts','titles','type', 'exact_answer']
    xtree = et.parse(xml_file)
    xroot = xtree.getroot()
    rows = []
    for question in xroot: 
        id = question.attrib.get("id")
        ir = question.find("IR")
        qp = question.find("QP")
        concepts = [e.text for e in qp.findall("Entities")]
        qa_type = qp.find("Type").text
        titles =  [e.find("Title").text for e in ir.findall("Result")]
        abstracts =  [e.find("Abstract").text for e in ir.findall("Result")]
        pmids = [e.get("PMID") for e in ir.findall("Result")]
        exact_answer = qa_answers[id] if id in qa_answers else None
        if DEBUG and not exact_answer:
            print(f"id [{id}] has no answer")
            no_answers +=1
        rows.append({"id":id,"human_concepts":concepts,"documents":pmids,"full_abstracts":abstracts,"titles":titles,"type":qa_type,'exact_answer':exact_answer})
    out_df = pd.DataFrame(rows, columns=df_cols)
    if DEBUG:
        print(f"{GREEN}[{no_answers}/{len(out_df)}]{OFF} {WHITE}questions had answers{OFF}")
    return out_df


In [361]:
def print_yes_no_info(df, tag):
    print(tag)
    print(f" [{len(df)}] {tag} Yes/No Questions")
    yes_df = df[df['exact_answer'] == 'yes']
    no_df = df[df['exact_answer'] == 'no']
    print(f" [{len(yes_df)}] {tag} Yes Questions")
    print(f" [{len(no_df)}] {tag} No Questions")


""" f1 Yes
    tp is gen 'yes' | gold 'yes'
    fp is gen 'yes' | gold 'no'
    fn is gen 'no' |  gold 'yes'

    f1 No
    tp is gen 'no' | gold 'no'
    fp is gen 'no' | gold 'yes'
    fn is gen 'yes' |  gold 'no'

    IGNORE if the predicted type is yes/no but gold type is different
"""
def do_yes_no_eval(gold_df,gen_df):
    print(f"{CYAN}Yes/No Evaluation{OFF}")
    yes_no_gold_df = gold_df[gold_df['type'] == 'yesno']
    yes_no_gen_df = gen_df[gen_df['type'] == 'yesno']
    
    if DEBUG:
        # Gold stats
        print_yes_no_info(yes_no_gold_df, "Gold")
        # Gen Stats
        print_yes_no_info(yes_no_gen_df, "Generated")

    gold_ids,gold_ans,gen_ids,gen_ans = get_col_list(yes_no_gold_df,gen_df,'exact_answer')

    # YES 
    ytp = 0
    yfp = 0
    yfn = 0
    
    for i in range (len(gold_ids)):
        gold_val = gold_ans[i]
        try:
            gen_val = gen_ans[gen_ids.index(gold_ids[i])]
        except ValueError as e:
            if DEBUG:
                print(e)
            continue
        if gen_val:
            if gold_val == 'yes':
                if gen_val =='yes':
                    ytp += 1
                elif gen_val =='no':
                    yfn += 1
                else:
                    if DEBUG:
                        print(f"yes question [{gold_ids[i]}] had generated answer {gen_val}")
            elif gold_val == 'no':
                if gen_val == 'yes':
                    yfp +=1
                elif gen_val =='no':
                    pass #handled by no f1
                else:
                    if DEBUG:
                        print(f"no question [{gold_ids[i]}] had generated answer {gen_val}")
            else:
                print(f"GOLDEN answer to yes/no question [{gold_ids[i]}] was {gold_val}")
                
        else: # not identified as yes/no question by generated
            pass
    #sanity check
    print (f"Yes | True Posative: {GREEN}{ytp}{OFF}, False Posative: {GREEN}{yfp}{OFF}, False Negative: {GREEN}{yfn}{OFF}")
    try:
        yp = ytp/(ytp + yfp)
    except:
        yp = 0
    try:
        yr = ytp/(ytp + yfn)
    except:
        yr = 0
    try:
        yf1 = 2 * ((yp * yr)/(yp+yr))
    except:
        yf1 = 0
    print (f'Yes | f1 {GREEN}{yf1}{OFF}, precision {GREEN}{yp}{OFF}, recall {GREEN}{yr}{OFF}')

    # NO SIDE
    ntp = 0
    nfp = 0
    nfn = 0

    for i in range (len(gold_ids)):
        gold_val = gold_ans[i]
        try:
            gen_val = gen_ans[gen_ids.index(gold_ids[i])]
        except ValueError as e:
            if DEBUG:
                print(e)
            continue
        if gen_val:
            if gold_val == 'no':
                if gen_val =='no':
                    ntp += 1
                elif gen_val =='yes':
                    nfn += 1
                else:
                    if DEBUG:
                        print(f"no question [{gold_ids[i]}] had generated answer {gen_val}")
            elif gold_val == 'yes':
                if gen_val == 'no':
                    nfp +=1
                elif gen_val =='yes':
                    pass #handled by no f1
                else:
                    print(f"yes question [{gold_ids[i]}] had generated answer {gen_val}")
            else:
                if DEBUG:
                    print(f"GOLDEN answer to yes/no question [{gold_ids[i]}] was {gold_val}")
                
        else: # not identified as yes/no question by generated
            pass
    
    # sanity check
    print (f"No | True Posative: {GREEN}{ntp}{OFF}, False Posative: {GREEN}{nfp}{OFF}, False Negative: {GREEN}{nfn}{OFF}")
    try:
        np = ntp/(ntp + nfp)
    except:
        np = 0
    try:
        nr = ntp/(ntp + nfn)
    except:
        nr = 0
    try:
        nf1 = 2 * ((np * nr)/(np+nr))
    except:
        nf1 = 0
    print (f'No | f1 {GREEN}{nf1}{OFF}, precision {GREEN}{np}{OFF}, recall {GREEN}{nr}{OFF}')

    f1 = (yf1 + nf1)/2 
    p = (yp + np)/2 
    r = (yr + nr)/2 
    print(f"Overall Yes/No | f1 {GREEN}{f1}{OFF}, precision {GREEN}{p}{OFF}, recall {GREEN}{r}{OFF}")
    print ('\n')
    return yf1,yp,yr,nf1,np,nr,f1,p,r

# yes_no_report = do_yes_no_eval(gold_df,gen_df)

In [362]:
# Compute [[Mean average precision, Geometric mean average precision]], precision, recall, f1 score
def do_concepts_eval(gold_df,gen_df):
    print(f"{CYAN}Concepts Evaluation{OFF}")
    gold_ids,gold_cons,gen_ids,gen_cons = get_col_list(gold_df,gen_df,'human_concepts')
    num_gen_q_without_cons = 0
    num_gold_q_without_cons = 0
    tp = 0
    fp = 0
    fn = 0

    scores = []
    # for each question
    for i in range (len(gold_ids)):
        gold_val = gold_cons[i]
        if not isinstance(gold_val,list) or gold_val == []:
            num_gold_q_without_cons += 1
            continue
        try:
            gen_val = gen_cons[gen_ids.index(gold_ids[i])]
        except ValueError as e:
            if DEBUG:
                print(e)
            continue
        # if concepts are found
        if gen_val != []:
            # TP is concept in Gold AND Gen
            # FP is concept NOT IN GOLD, but YES IN GEN
            # FN is concept IN Gold but NOT GEN

            # get unique concepts from both gold and gen
            unique_gold_cons = set(gold_val[0])
            unique_gen_cons = set(gen_val[0])
            for val in unique_gold_cons:
                if val in unique_gen_cons:
                    tp += 1
                elif val not in unique_gen_cons:
                    fn += 1
            for val in unique_gen_cons:
                if val not in unique_gold_cons:
                    fp += 1

            f1,p,r = get_f1_p_r(tp,fp,fn, tag ="Concepts")
            scores.append((f1,p,r))
        else: # There are no concepts retrieved for this document
            num_gen_q_without_cons += 1
            pass
    #sanity check
    print(f"{GREEN}[{len(gold_ids) - num_gold_q_without_cons}/{len(gold_ids)}]{OFF} Questions have human readable concepts in gold dataset")
    print(f"{GREEN}[{len(gen_ids) - num_gen_q_without_cons}/{len(gen_ids)}]{OFF} Questions have human readable concepts in generated dataset")

    # OVERALL SCORES
    f1_sum = p_sum = r_sum = 0
    for f1,p,r in scores:
        f1_sum += f1
        p_sum += p
        r_sum += r
    f1_sum /= len(scores)
    p_sum /= len(scores)
    r_sum /= len(scores)

    print (f'Concepts mean f1 {GREEN}{f1_sum}{OFF}, precision {GREEN}{p_sum}{OFF}, recall {GREEN}{r_sum}{OFF}')
    print ('\n')
    return f1_sum,p_sum,r_sum,scores

# concepts_report = do_concepts_eval(gold_df,gen_df)

In [363]:
def get_f1_p_r(tp,fp,fn, tag ="calculated"):
    if DEBUG:
        print (f"{tag} tp: {tp}, fp: {fp}, fn: {fn}")
    try:
        p = tp/(tp + fp)
    except:
        p = 0
    try:
        r = tp/(tp + fn)
    except:
        r = 0
    try:
        f1 = 2 * ((p * r)/(p+r))
    except:
        f1 = 0
    if DEBUG:
        print (f'{tag} f1 {f1}, precision {p}, recall {r}')
    return f1,p,r

In [364]:
def do_pmids_eval(gold_df,gen_df):
    print(f"{CYAN}PubMed Documents Evaluation{OFF}")
    # pmids are the pubmed document ids
    gold_ids,gold_pmids,gen_ids,gen_pmids = get_col_list(gold_df,gen_df,'documents')
    num_gen_q_without_docs = 0
    num_gold_q_without_docs = 0
    tp = 0
    fp = 0
    fn = 0

    scores = []
    # for each question
    for i in range (len(gold_ids)):
        gold_val = gold_pmids[i]
        if gold_val == []:
            num_gold_q_without_docs += 1
            continue
        try:
            gen_val = gen_pmids[gen_ids.index(gold_ids[i])]
        except ValueError as e:
            if DEBUG:
                print(e)
            continue
        # if documents are found
        print(gen_val,type(gen_val))
        if isinstance(gen_val, list) and gen_val != []:
            # TP is pmid in Gold AND Gen
            # FP is pmid NOT IN GOLD, but YES IN GEN
            # FN is pmid IN Gold but NOT GEN

            # get unique PMIDs from both gold and gen
            try:
                unique_gold_pmids = set(gold_val[0])
                unique_gen_pmids = set(gen_val[0])
            except Exception as e:
                continue
            for val in unique_gold_pmids:
                if val in unique_gen_pmids:
                    tp += 1
                elif val not in unique_gen_pmids:
                    fn += 1
            for val in unique_gen_pmids:
                if val not in unique_gold_pmids:
                    fp += 1

            f1,p,r = get_f1_p_r(tp,fp,fn, tag ="PubMed Documents")
            scores.append((f1,p,r))
        else: # There are no documents retrieved for this document
            num_gen_q_without_docs += 1
            pass
    #sanity check
    print(f"{GREEN}[{len(gold_ids) - num_gold_q_without_docs}/{len(gold_ids)}]{OFF} Questions have documents in gold dataset")
    print(f"{GREEN}[{len(gen_ids) - num_gen_q_without_docs}/{len(gen_ids)}]{OFF} Questions have documents in generated dataset")

    # OVERALL SCORES
    f1_sum = p_sum = r_sum = 0
    for f1,p,r in scores:
        f1_sum += f1
        p_sum += p
        r_sum += r
    f1_sum /= len(scores)
    p_sum /= len(scores)
    r_sum /= len(scores)

    print (f'PubMed Documents mean f1 {GREEN}{f1_sum}{OFF}, precision {GREEN}{p_sum}{OFF}, recall {GREEN}{r_sum}{OFF}')
    print ('\n')
    return f1_sum,p_sum,r_sum,scores

# pmid_report = do_pmids_eval(gold_df,gen_df)

In [365]:
# We use strict and leniant accuracy  (first result, or any result)
def do_factoid_eval(gold_df,gen_factoid_path):
    print(f"{CYAN}Factoid Evaluation{OFF}")
    factoid_gold_df = gold_df[gold_df['type'] == 'factoid']
    factoid_gen_df = gen_df[gen_df['type'] == 'factoid']

    if DEBUG:
        print(f" [{len(factoid_gold_df)}] Gold Factoid Questions")
        print(f" [{len(factoid_gen_df)}] Generated Factoid Questions")
    gold_ids,gold_ans,gen_ids,gen_ans = get_col_list(factoid_gold_df,gen_df,'exact_answer')

    # Use alternative strategy to handle ranked factoid preds
    with open(gen_factoid_path, "r") as ft_file:
        factoid_gen_json = json.load(ft_file)
    
    gen_factoid_answers = {}
    for question in factoid_gen_json["questions"]:
        id = question["id"]
        if len(id) == 24:
            id = id[0:20]
        answer = question['exact_answer']
        if isinstance(answer,list): 
            if isinstance(answer[0],list): # handle list in list
                answer = [e[0] for e in answer]
        gen_factoid_answers[id] = answer

    num_gold_q_without_ans = 0
    num_strict = 0
    num_leniant = 0
    num_total = 0
    mrrs = []
    # for each question
    for i in range (len(gold_ids)):
        gold_val = gold_ans[i][0]
        if gold_val == []:
            num_gold_q_without_ans += 1
            continue
        # trim last 4 digits which get removed for the final bioasq form answers
        trimmed_id = gold_ids[i][0:20]
        if trimmed_id not in gen_factoid_answers.keys():
            if DEBUG:
                print(f"{trimmed_id} wasn't correctly identified as factoid")
            continue
        gen_vals = gen_factoid_answers[trimmed_id]
        gen_vals_clean = [e.lower().strip() for e in gen_vals]
        if DEBUG:
            print(gold_val," | ",gen_vals)
        # accuracy calculations
        gold_val_clean = gold_val
        num_total += 1
        if gold_val_clean == gen_vals_clean[0]: # force lowercase / strip whitespace to help
            num_strict += 1
            num_leniant += 1
        elif gold_val_clean in gen_vals_clean:
            num_leniant += 1

        # mrr calculations
        mrr = 0
        r = 0
        n = len(gen_vals_clean)
        for i in range(1,n+1):
            if gen_vals_clean[i-1] == gold_val_clean:
                r = i
                break
        if r != 0:
            mrr = 1/n * 1/r 
            if DEBUG:
                print(f"{trimmed_id} MRR: {mrr}")
        mrrs.append(mrr)

    average_mrr = sum(mrrs) / len(mrrs)
    leniant_acc = num_leniant/num_total
    strict_acc = num_strict/num_total

    # sanity check
    print(f"{GREEN}[{len(gold_ids) - num_gold_q_without_ans}/{len(gold_ids)}]{OFF} Factoid questions have answers in gold dataset")
    print(f"{GREEN}[{num_total}/{len(gen_factoid_answers)}]{OFF} Factoid questions have answers in generated dataset")
    print(f"Leniant Accuracy: {GREEN}{leniant_acc}{OFF}, Strict Accuracy: {GREEN}{strict_acc}{OFF}, Mean Reciprocal Rank (MRR): {GREEN}{average_mrr}{OFF}")
    print ('\n')
    return leniant_acc,strict_acc,average_mrr,mrrs

# do_factoid_eval(gold_df,'tmp/qa/factoid/BioASQform_BioASQ-answer.json')

In [366]:
def do_list_eval(gold_df,gen_df):
    print(f"{CYAN}List Evaluation{OFF}")
    list_gold_df = gold_df[gold_df['type'] == 'list']
    list_gen_df = gen_df[gen_df['type'] == 'list']

    if DEBUG:
        print(f" [{len(list_gold_df)}] Gold List Questions")
        print(f" [{len(list_gen_df)}] Generated List Questions")
    
    gold_ids,gold_ans,gen_ids,gen_ans = get_col_list(list_gold_df,gen_df,'exact_answer')
    num_gen_q_without_ans = 0
    num_gold_q_without_ans = 0
    tp = 0
    fp = 0
    fn = 0

    num_gen = 0
    scores = []
    # for each question
    for i in range (len(gold_ids)):
        gold_val = gold_ans[i]
        if gold_val == []:
            num_gold_q_without_ans += 1
            continue
        try:
            gen_val = gen_ans[gen_ids.index(gold_ids[i])]
        except ValueError as e:
            if DEBUG:
                print(e)
            continue
        # if answers are found
        if gen_val != None: # List is only able to find a single item list
            # TP is answer in Gold AND Gen
            # FP is answer NOT IN GOLD, but YES IN GEN
            # FN is answer IN Gold but NOT GEN
            gold_list = gold_val[0]
            for val in gold_list:
                if val in gen_val:
                    tp += 1
                elif val not in gen_val:
                    fn += 1
            for val in gen_val:
                if val not in gold_list:
                    fp += 1
            num_gen += 1
            f1,p,r = get_f1_p_r(tp,fp,fn, tag ="List Questions")
            scores.append((f1,p,r))
    #sanity check
    print(f"{GREEN}[{len(gold_ids) - num_gold_q_without_ans}/{len(gold_ids)}]{OFF} List questions have answers in gold dataset")
    print(f"{GREEN}[{num_gen}/{len(list_gen_df)}]{OFF} List questions have answers in generated dataset")

    # OVERALL SCORES
    f1_sum = p_sum = r_sum = 0
    for f1,p,r in scores:
        f1_sum += f1
        p_sum += p
        r_sum += r
    f1_sum /= len(scores)
    p_sum /= len(scores)
    r_sum /= len(scores)

    print (f'List Questions mean f1 {GREEN}{f1_sum}{OFF}, precision {GREEN}{p_sum}{OFF}, recall {GREEN}{r_sum}{OFF}')
    print ('\n')
    return f1_sum,p_sum,r_sum,scores

# list_report = do_list_eval(gold_df,gen_df)


In [367]:
def gen_gold_qu_output(gold_df, gen_folder, gen_xml_name="bioasq_qa.xml"):
    qu_generated = gen_folder + "/ir/input/" + gen_xml_name
    new_file_name = qu_generated.replace(".xml", "_GOLD.xml")

    fileTree = et.parse(qu_generated)
    if fileTree:
        root = fileTree.getroot()
        questions = root.findall('Q')
        for question in questions:
            id = question.attrib.get("id")
            qp = question.find("QP")
            # remove type and entities
            qp.clear()
            # type is the fifth element in the row
            gold_type = gold_df.loc[gold_df['id'] == id].values[0][4]
            type_ele = et.SubElement(qp,'Type')
            type_ele.text = gold_type
            gold_concepts = gold_df.loc[gold_df['id'] == id].values[0][7]
            ent_list = []
            qp_query = et.SubElement(qp,'Query')
            if not isinstance(gold_concepts,list):
                if DEBUG:
                    print(f"Question [{id}] has no golden human_concepts [{type(gold_concepts)}]")
                continue
            for ent in gold_concepts:
                ent_list.append(str(ent))
                qp_en = et.SubElement(qp,'Entities') 
                qp_en.text = str(ent)
            qp_query.text = str(' '.join(ent_list))
        tree = et.ElementTree(root)
        os.makedirs(os.path.dirname(new_file_name), exist_ok=True)
        print(f"Writing gold QU output / IR input to {new_file_name}")
        tree.write(new_file_name, pretty_print=True)
    return new_file_name


In [375]:
def gen_gold_ir_output(gold_df, gen_folder, gen_xml_name="bioasq_qa.xml"):
    ir_generated = gen_folder + "/ir/output/" + gen_xml_name
    new_file_name = ir_generated.replace(".xml", "_GOLD.xml")

    fileTree = et.parse(ir_generated)
    if fileTree:
        root = fileTree.getroot()
        questions = root.findall('Q')
        for question in questions:
            id = question.attrib.get("id")
            original_question = question.text
            if DEBUG:
                print(original_question)
            ir = question.find("IR")
            # remove original generated articles
            ir.clear()

            gold_abstracts = gold_df.loc[gold_df['id'] == id].values[0][8]
            gold_titles = gold_df.loc[gold_df['id'] == id].values[0][9]
            # system just using top abstract atm
            if isinstance(gold_abstracts,list):
                gold_abstract = gold_abstracts[0]
            else:
                gold_abstract = ""
            if isinstance(gold_titles,list):
                gold_title = gold_titles[0]
            else:
                gold_title = ""
            # fill result
            result_tag = et.SubElement(ir, "Result")
            gold_df.loc[gold_df['id'] == id].values[0][8]
            pmid = gold_df.loc[gold_df['id'] == id].values[0][1][0]
            result_tag.set("PMID", pmid)
            title = et.SubElement(result_tag, "Title")
            title.text = gold_title
            abstract = et.SubElement(result_tag, "Abstract")
            abstract.text = gold_abstract
        tree = et.ElementTree(root)
        os.makedirs(os.path.dirname(new_file_name), exist_ok=True)
        print(f"Writing gold QA input / IR output to {new_file_name}")
        tree.write(new_file_name, pretty_print=True)
    return new_file_name

In [369]:
def generate_intermediary_datasets(gold_df,qu_output,ir_output):
    # QU: replace concepts with human_concepts and type with gold type
    gold_qu_output = gen_gold_qu_output(gold_df,qu_output)
    # IR: Replace IR with full_abstract in document format
    gold_ir_output = gen_gold_ir_output(gold_df,ir_output)

    return gold_qu_output, gold_ir_output

In [370]:
def run_all_the_tests(gold_dataset_path, generation_folder_path, gen_xml_name):
    factoid_path = generation_folder_path + "/qa/factoid/BioASQform_BioASQ-answer.json"
    generated_qu = generation_folder_path + "/ir/output/" + gen_xml_name

    with open(gold_dataset_path, "r") as f:
        gold_data = json.loads(f.read())
    # load and flatten data
    gold_df = pd.json_normalize(gold_data, record_path="questions")
    # get gold df
    gold_df["documents"] = gold_df["documents"].apply(get_pmid)
    # get generated df
    gen_df = parse_xml(generated_qu, generation_folder_path + "/qa")

    # Concepts
    concepts_report = do_concepts_eval(gold_df, gen_df)
    # print(concepts_report)

    # Documents
    pmids_report = do_pmids_eval(gold_df, gen_df)
    # print(pmids_report)

    # Type
    gold_type = gold_df["type"].to_numpy()
    gen_type = gen_df["type"].to_numpy()

    try:
        print(f"{CYAN}Type Evaluation{OFF}")
        print(classification_report(gold_type, gen_type))
        type_report = classification_report(gold_type, gen_type, output_dict=True)
    except:
        type_report = f"TypeReport: Found input variables with inconsistent numbers of samples: [{len(gold_type)}] [{len(gen_type)}]"
    # print(type_report)

    # Yes/No Question Answering
    yes_no_report = do_yes_no_eval(gold_df, gen_df)
    # print(yes_no_report)

    # Factoid Question Answering
    factoid_report = do_factoid_eval(gold_df, factoid_path)
    # print(factoid_report)

    # List Question Answering
    list_report = do_list_eval(gold_df, gen_df)
    # print(list_report)

    return (
        concepts_report,
        pmids_report,
        type_report,
        yes_no_report,
        factoid_report,
        list_report,
    )


In [371]:
def save_results(test_results,save_path):
    t = time.localtime()
    date = time.strftime('%b-%d-%Y', t)
    concepts_report,pmids_report,type_report,yes_no_report,factoid_report,list_report = test_results
    results_folder = save_path + "/test_results/" + date 
    timestamp = time.strftime('%H%M%S', t)
    if not os.path.exists(results_folder):
        os.makedirs(results_folder)

    # Save concepts
    con_name = results_folder + f"/concepts-{timestamp}.csv"
    with open(con_name,'w+') as f:
        f1_sum,p_sum,r_sum,scores = concepts_report
        f.write("Average f1 score,Average precision,Average Recall\n")
        f.write(f"{f1_sum},{p_sum},{r_sum}\n")
        f.write("f1,precision,recall\n")
        for f1,p,r in scores:
            f.write(f"{f1},{p},{r}\n")
    print(f"Saved concepts info to {con_name}")

    # Save pmids
    pmid_name = results_folder + f"/pmids-{timestamp}.csv"
    with open(pmid_name,'w+') as f:
        f1_sum,p_sum,r_sum,scores = pmids_report
        f.write("Average f1 score,Average precision,Average Recall\n")
        f.write(f"{f1_sum},{p_sum},{r_sum}\n")
        f.write("f1,precision,recall\n")
        for f1,p,r in scores:
            f.write(f"{f1},{p},{r}\n")
    print(f"Saved pmid info to {pmid_name}")


    # Save type report
    type_name = results_folder + f"/type-{timestamp}.json"
    with open(type_name,'w+') as f:
        json.dump(type_report,f,indent=2)
    print(f"Saved type info to {type_name}")


    # Save yes/no
    yesno_name = results_folder + f"/yesno-{timestamp}.csv"
    with open(yesno_name,'w+') as f: 
        yf1,yp,yr,nf1,np,nr,f1,p,r = yes_no_report
        f.write("Yes/No f1 score,Yes/No precision ,Yes/No recall\n")
        f.write(f"{f1},{p},{r}\n")
        f.write("Yes f1 score,Yes precision ,Yes recall\n")
        f.write(f"{yf1},{yp},{yr}\n")
        f.write("No f1 score,No precision ,No recall\n")
        f.write(f"{nf1},{np},{nr}\n")
    print(f"Saved yes/no info to {yesno_name}")

    # Save factoids
    fact_name = results_folder + f"/factoid-{timestamp}.csv"
    with open(fact_name,'w+') as f:
        leniant_acc,strict_acc,average_mrr,mrrs = factoid_report
        f.write("Factoid leniant accuracy,Factoid strict accuracy,Factoid average mrr\n")
        f.write(f"{leniant_acc},{strict_acc},{average_mrr}\n")
        f.write("mrr\n")
        for mrr in mrrs:
            f.write(f"{mrr}\n")
    print(f"Saved factoid info to {fact_name}")

    # Save list
    list_name = results_folder + f"/list-{timestamp}.csv"
    with open(list_name,'w+') as f:
        f1_sum,p_sum,r_sum,scores = list_report
        f.write("Average f1 score,Average precision,Average Recall\n")
        f.write(f"{f1_sum},{p_sum},{r_sum}\n")
        f.write("f1,precision,recall\n")
        for f1,p,r in scores:
            f.write(f"{f1},{p},{r}\n")
    print(f"Saved list info to {list_name}")



In [372]:
# Set up the golden answer dataframe
DEBUG = False
golden_dataset_path = "testing_datasets/augmented_concepts_abstracts_titles.json"
# gen_folder = "tmp/small_batch"
gen_folder = "tmp"

# xml_name = "bioasq_qa_SMALL.xml"
xml_name = "bioasq_qa.xml"

test_results = run_all_the_tests(golden_dataset_path,gen_folder,xml_name)
save_results(test_results,gen_folder)


Concepts Evaluation
[1607/3243] Questions have human readable concepts in gold dataset
[3242/3243] Questions have human readable concepts in generated dataset
Concepts mean f1 0.507613033876486, precision 0.6096832682725072, recall 0.43495543509698553


PubMed Documents Evaluation
[] <class 'list'>
['31913761', '31913761', '31913761', '31913761', '31913761'] <class 'list'>
[] <class 'list'>
['8107241', '32380211', '32380211', '32380211', '32380211'] <class 'list'>
['31323160', '31323160', '31323160', '31323160', '14769514'] <class 'list'>
[] <class 'list'>
['30793226', '30793226'] <class 'list'>
['31604651', '25310725', '25310725', '25310725', '32337145'] <class 'list'>
[] <class 'list'>
['15870890'] <class 'list'>
['1433123', '31419189'] <class 'list'>
[] <class 'list'>
[] <class 'list'>
['31345557', '31345557', '31345557', '31345557', '31345557'] <class 'list'>
['24799061', '32223238', '32223238', '32223238', '22987974'] <class 'list'>
[] <class 'list'>
['32370214', '32370214', '3005

In [373]:
# gen_gold_qu_output(gold_df,"tmp/small_batch/ir/input/bioasq_qa_SMALL.xml")
# gen_gold_ir_output(gold_df,"tmp/small_batch/ir/output/bioasq_qa_SMALL.xml")

generate_intermediary_datasets(gold_df,"tmp/small_batch/ir/input/bioasq_qa_SMALL.xml","tmp/small_batch/ir/output/bioasq_qa_SMALL.xml")

OSError: Error reading file 'tmp/small_batch/ir/input/bioasq_qa_SMALL.xml/ir/input/bioasq_qa.xml': failed to load external entity "tmp/small_batch/ir/input/bioasq_qa_SMALL.xml/ir/input/bioasq_qa.xml"

In [376]:
def get_gold_df(gold_dataset_path):
    with open(gold_dataset_path, "r") as f:
        gold_data = json.loads(f.read())
    # load and flatten data
    gold_df = pd.json_normalize(gold_data, record_path="questions")
    # get gold df
    gold_df["documents"] = gold_df["documents"].apply(get_pmid)
    return gold_df
# Run system raw
golden_dataset_path = "testing_datasets/augmented_concepts_abstracts_titles.json"
gen_folder = "tmp/small_batch"
xml_name = "bioasq_qa_SMALL.xml"
gold_df =  get_gold_df(golden_dataset_path)
#gold_ir_output  = gen_gold_qu_output(gold_df,gen_folder,xml_name)
gen_gold_ir_output(gold_df,gen_folder,xml_name)
xml_name = "bioasq_qa_SMALL_GOLD.xml"
gold_qu_ir_test_results = run_all_the_tests(golden_dataset_path, gen_folder,xml_name)

Writing gold QA input / IR output to tmp/small_batch/ir/output/bioasq_qa_SMALL_GOLD.xml
Concepts Evaluation
[1607/3243] Questions have human readable concepts in gold dataset
[11/11] Questions have human readable concepts in generated dataset
Concepts mean f1 0.6192845811680302, precision 0.6548998838145618, recall 0.5901720164334147


PubMed Documents Evaluation
['15858239'] <class 'list'>
['23959273'] <class 'list'>
['3320045'] <class 'list'>
['22955988'] <class 'list'>
['22948539'] <class 'list'>
['26191653'] <class 'list'>
['21345725'] <class 'list'>
['21815707'] <class 'list'>
['24126422'] <class 'list'>
['12883684'] <class 'list'>
['3819049'] <class 'list'>
[3243/3243] Questions have documents in gold dataset
[11/11] Questions have documents in generated dataset
PubMed Documents mean f1 1.0, precision 1.0, recall 1.0


Type Evaluation
Yes/No Evaluation
Yes | True Posative: 0, False Posative: 0, False Negative: 2
Yes | f1 0, precision 0, recall 0.0
No | True Posative: 0, False Pos